In [1]:
import os 
import json
import pandas as pd
import spacy
import re
import string
from collections import defaultdict
import warnings
import numpy as np
import random

warnings.filterwarnings('ignore')

2024-03-28 19:57:17.870834: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 19:57:17.908865: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 19:57:19.106976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Task 1

#### Convert the Baseline unsummarized output data to model needs format
- subtask1: input from baseline json marker
/users/the/NER_MTB/subtask1/Gold_PairWise_EventTimex/breast/breast_patient02_test/patient02_report023_NOTE/patient02_report023_NOTE.Temporal_Relation.gold.entity_only.xml
- subtask2: input raw

In [3]:
cancer = "breast"
test_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_test_output.tsv"
gold_dct_file = f"/users/the/NER_MTB/0_{cancer}_test_gold_dct.json"
# here has been replace /n to space
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_test", "Patient_Notes", f"{cancer}", f"test")
unsummarized_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_test_output.tsv"
df = pd.read_csv(unsummarized_file, delimiter="\t")

def find_loc(a, b):
    # a: long one
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)
            
        if s == index + len(b_clean):
            out.append(i)
    if len(out) == 2:
        end, start = out[0], out[1]+1
        return end, start
    else: 
        return None, None
    
def _get_seindexs_single_note(string, spacy_better):
    """Return the inedx of sent and sent_lst after doc"""
    doc = spacy_better(string)
    seindexs_single_note = [] 
    sent_lst = []
    for i, sent in enumerate(doc.sents):
        seindexs_single_note.append((i, sent.start_char, sent.end_char))
        sent_lst.append(sent.text)
    return seindexs_single_note, sent_lst

def _find_nthind(tuple_list, item_start_ind, num_pre9_tokens):
    for tup in tuple_list:
        if tup[1] <= (int(item_start_ind)-num_pre9_tokens) <= tup[2]:
            return tup[0]
    print("something wrong with this case") # TO-DO: need to fix here, run the below code first since little error msgs
    return None  # Return None if no match is found # across the sentences

def _find_surrounding_text(string):
    tlink_inst = string
    e_pattern = r'<e>\s*(.*?)\s*</e>'
    matches = re.findall(e_pattern, tlink_inst)
    matchess = re.finditer(e_pattern, tlink_inst)
    for match in matchess:
        matched_text = match.group(1)  # Get the matched text
        start_index = match.start(1)  # Get the start index of the matched text
    return matches, start_index 


def _find_time_surrounding_text(string):
    tlink_inst = string
    t_pattern = r'<t>\s*(.*?)\s*</t>'
    matches = re.findall(t_pattern, tlink_inst)
    matchess = re.finditer(t_pattern, tlink_inst)
    for match in matchess:
        matched_text = match.group(1)  # Get the matched text
        start_index = match.start(1)  # Get the start index of the matched text
    return matches, start_index

def _find_time_word_index(original_text, word):
    print("wrapped time: ", word)
    original_text_processed = original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    word = word.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    start, end = find_loc(original_text_processed, word)
    return start, end

def _find_chemo_word_index(original_text,word, tlink_inst, tagger_before_ind):
    togger_before_ind = tagger_before_ind-5
    longer_before_in_tlink = tlink_inst[:tagger_before_ind] # remove <e>
    longer_before_in_tlink = longer_before_in_tlink.replace("<t>", "")
    longer_before_in_tlink = longer_before_in_tlink.replace("</t>", "")
    longer_before_in_tlink = longer_before_in_tlink.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    original_text_processed = original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    longer_before_in_tlink = longer_before_in_tlink[:-3]
    _, tlink_tagger_before_ind = find_loc(original_text_processed, longer_before_in_tlink)

    word = word.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # print("word: ", word)
    print(type(tlink_tagger_before_ind), tlink_tagger_before_ind)
    chunked_original_text = original_text[tlink_tagger_before_ind:]
    original_after_procesed = chunked_original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # temp_start, temp_end = find_loc(original_after_procesed, word)
    temp_start, temp_end = find_loc(original_after_procesed, word)
  
    if temp_start is None or tlink_tagger_before_ind is None or temp_end is None or tlink_tagger_before_ind is None: 
        start, end = None, None 
        return None, None 
    else: 
        start, end = temp_start+tlink_tagger_before_ind , temp_end+tlink_tagger_before_ind 
        return start, end 


def _find_nth(lst, chemo_or_timex):
    "Used the loc from raw to find where sent it is in Doc"
    nth_sent_lst = []
    if len(lst)!=0:
        for i, chemo in enumerate(lst):
            if chemo_or_timex == "chemo":
                item_start_ind, item_end_ind = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][i][0], gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][i][1]
            else:
                item_start_ind, item_end_ind = gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][i][0], gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][i][1]
            # print(item_start_ind, item_end_ind)
            for sent_id, sent in enumerate(doc.sents):
                # print(sent.start_char, sent.end_char)
                if sent.start_char <= item_start_ind-num_pre9_tokens< sent.end_char: 
                    for token in sent:
                        if token.idx <= item_start_ind-num_pre9_tokens< (token.idx+len(token.text)):
                            print(f"{chemo_or_timex} Token:", token)
                            print(f"{chemo_or_timex} Sentence:", sent)
                            print(f"{chemo_or_timex} Sentence ID:", sent_id)
                            nth_sent_lst.append(sent_id)
    assert len(lst) == len(nth_sent_lst)
    return nth_sent_lst

def _find_tsv_sentences_in_range(doc, start_index, end_index):
    "Given nth sents of chemo and timex, return the context"
    sentences_in_range = []
    for ind, sent in enumerate(doc.sents):
        if ind >= start_index and ind < end_index:
            sentences_in_range.append(sent)
    return sentences_in_range

def _find_sentences_in_range(doc, start_index, end_index):
    "Given nth sents of chemo and timex, return the context"
    sentences_in_range = []
    for ind, sent in enumerate(doc.sents):
        if ind == start_index:
            sentences_in_range.append(sent.text)
            ind += 1
            while ind < end_index:
                sentences_in_range.append(list(doc.sents)[ind].text)
                ind += 1
            sentences = " ".join(sentences_in_range)
            return sentences

def _df_find_nth(lst, match_start_lst, match_end_lst):
    "Used the loc from raw to find where sent it is in Doc"
    nth_sent_lst = []
    if len(lst)!=0:
        for i, chemo in enumerate(lst):
            item_start_ind, item_end_ind = match_start_lst[i], match_end_lst[i]
            if item_end_ind is None or item_start_ind is None:
                nth_sent_lst.append(None)
            else:
                for sent_id, sent in enumerate(doc.sents):
                    # print(sent.start_char, sent.end_char)
                    if sent.start_char <= item_start_ind< sent.end_char: 
                        # print("chemo_range: ", sent_id, item_start_ind, (sent.start_char, sent.end_char))
                        for token in sent:
                            if token.idx <= item_start_ind< (token.idx+len(token.text)):
                                # print("Token:", token)
                                # print("Sentence:", sent)
                                # print("Sentence ID:", sent_id)
                                nth_sent_lst.append(int(sent_id))
    assert len(lst) == len(nth_sent_lst)
    return nth_sent_lst

with open(gold_dct_file, "r") as infile: 
    gold_dct = json.load(infile)


spacy_better = spacy.load("en_core_web_sm")

pos_rel = []
final_df = pd.DataFrame()
cnt = 0
skip = 0

# for pat_id in ["patient47"]:
# for pat_id in gold_dct.keys():
    note_sub_dir = os.path.join(note_dir, pat_id)
    medium_df = pd.DataFrame()
    for filename in gold_dct[pat_id].keys():
    # for filename in ["report030_NOTE"]:

        # raw_note = gold_dct[pat_id][filename]["raw_note"]
        pat_filename = f"{pat_id}_{filename}.txt"
        raw_note_file = os.path.join(note_sub_dir, pat_filename)
        with open(raw_note_file, "r") as infile: 
            lines = infile.readlines()
        num_pre9_tokens = len(''.join(lines[:9]))
        lines = lines[9:]
        clean_note = "".join(lines)
        doc = spacy_better(clean_note)
        # print(f"len(doc.sents), ", len(list(doc.sents)))

        chemo_lst, timex_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment"]
        
        # print(chemo_lst, timex_lst)
        chemo_nth_sent = _find_nth(chemo_lst, "chemo")
        timex_nth_sent = _find_nth(timex_lst, "timex")
        
        gold_dct[pat_id][filename]["pair_wise"]["chemo"]["nth_sent"] = chemo_nth_sent
        gold_dct[pat_id][filename]["pair_wise"]["time"]["nth_sent"] = timex_nth_sent
        gold_dct[pat_id][filename]["num_pre9_tokens"] = num_pre9_tokens




        # create positive instance
        gold_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["rel_id"]
        if len(gold_pair) == 0:
            continue
        gold_sid_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["source_id"]
        gold_tid_pair = gold_dct[pat_id][filename]["chemo_time_rel"]["target_id"]

        num_pre9_tokens = gold_dct[pat_id][filename]["num_pre9_tokens"]

        rel_type_lst = gold_dct[pat_id][filename]["chemo_time_rel"]["rel_type"]

        chemo_ment_lst, time_ment_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment"]
        chemo_span_lst, time_span_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"], gold_dct[pat_id][filename]["pair_wise"]["time"]["span"]
        chemo_nth_sent_lst, time_nth_sent_lst = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["nth_sent"], gold_dct[pat_id][filename]["pair_wise"]["time"]["nth_sent"]
        
        chemo_id, time_id = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"]
        combination = []
        for i in range(len(gold_pair)):
            pos_ins = defaultdict(dict)
            source_id, target_id = gold_sid_pair[i], gold_tid_pair[i]
            if source_id in chemo_id: 
                chemo_ind = chemo_id.index(source_id) 
                chemo_ment = chemo_ment_lst[chemo_ind]
                chemo_start, chemo_end = chemo_span_lst[chemo_ind][0], chemo_span_lst[chemo_ind][1]
                chemo_nth_sent = chemo_nth_sent_lst[chemo_ind]
                if target_id not in time_id:
                    print(pat_id, filename)
                time_ind = time_id.index(target_id)
                time_ment = time_ment_lst[time_ind]
                time_start, time_end = time_span_lst[time_ind][0], time_span_lst[time_ind][1]
                time_nth_sent = time_nth_sent_lst[time_ind]
            else: 
                time_ind = time_id.index(source_id) 
                time_ment = time_ment_lst[time_ind]
                time_start, time_end = time_span_lst[time_ind][0], time_span_lst[time_ind][1]
                time_nth_sent = time_nth_sent_lst[time_ind]
                chemo_ind = chemo_id.index(target_id) 
                chemo_ment = chemo_ment_lst[chemo_ind]
                chemo_start, chemo_end = chemo_span_lst[chemo_ind][0], chemo_span_lst[chemo_ind][1]
                chemo_nth_sent = chemo_nth_sent_lst[chemo_ind]

            pos_ins["filename"] = f"{pat_id}_{filename}"
            pos_ins["statement"] = "Does the arg1 and arg2 has relationship?"
            pos_ins["num_pre9_tokens"] = num_pre9_tokens
            pos_ins["rel_type"] = rel_type_lst[i] 
            pos_ins["label"] = "positive"
            pos_ins["arg1"] = {
                "mention":chemo_ment,
                "start_paper_ind":chemo_start,
                "end_paper_ind":chemo_end,
                "nth_sent":chemo_nth_sent
            }
            pos_ins["arg2"] = {
                "mention":time_ment,
                "start_paper_ind":time_start,
                "end_paper_ind":time_end,
                "nth_sent":time_nth_sent
            }
            if int(chemo_nth_sent) > int(time_nth_sent):
                start_nth_sent = time_nth_sent
                end_nth_sent = chemo_nth_sent
            else: 
                start_nth_sent = chemo_nth_sent
                end_nth_sent = time_nth_sent
            pos_ins["context"] = _find_sentences_in_range(doc, start_nth_sent, end_nth_sent+1) 
            # " ".join([sent.text for sent in row["match_sents"]])
        
            pos_rel.append(pos_ins)

'''

'''
test_all_patient_id = ""
        # 2
        chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst = [], []
        nth_sent, chemo_lst= [], []
        time_start_ind_tlinkinst, time_end_ind_tlinkinst = [], []
        time_nth_sent,raw_time_lst = [], []
        
        match_sents = []
        
        patient_file = df.loc[df["note_name"] == pat_id + "_" + filename]

        tsv_df = pd.DataFrame()
        for i, row in patient_file.iterrows():
            tlink_inst = row["tlink_inst"]
            norm_time = row["normed_timex"]
            if (tlink_inst != "none") and norm_time[0]!= "P": #why here
                matches, before_tag_ind = _find_surrounding_text(tlink_inst)

                print(pat_id, filename)

                time_matches, _ = _find_time_surrounding_text(tlink_inst)
                # if pat_id == "patient47" and filename== "report031_NOTE" and time_matches[0]=="today 's":
                #     continue
                # elif time_matches[0]=="daily" and matches=="chemotherapy":
                #     continue
                # elif time_matches[0]=="every 12 hours":
                #     continue
                # elif pat_id == "patient47" and filename== "report031_NOTE" and time_matches[0]=="one hour prior" and matches=="chemotherapy":
                #     continue
                # elif pat_id == "patient47" and filename== "report040_NOTE" and time_matches[0]=="currently" and matches=="Taxol":
                #     continue
                # elif pat_id == "patient47" and filename== "report040_NOTE" and time_matches[0]=="currently" and matches=="chemotherapy":
                #     continue
                # # elif pat_id == "patient47" and filename== "report030_NOTE" and time_matches[0]=="today 's":
                # #     continue
                # # elif pat_id == "patient47" and filename== "report030_NOTE" and time_matches[0]=="today 's" and matches=="Cytoxan":
                #     continue
                # elif time_matches[0] == "three times a day":
                #     continue
                # elif time_matches[0] == "one day prior":
                #     continue
                # elif time_matches[0] == "2 weeks after":
                #     continue


                # print("1: ", matches, time_matches[0])

                
                # if print out, issue with your code
                # if len(matches) == 0 or len(time_matches) == 0:
                #     if len(matches) == 0:
                #         print("chemo: yese", matches)
                #     elif len(time_matches) == 0:
                #         print("1time: ")

                chemo_lst.append(matches[0])
                print("matched chemo: ", matches[0])
                raw_time_lst.append(time_matches[0]) 
                print("matched time: ", time_matches[0])

                # clean_match = matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                # time_clean_match = time_matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                
                # print(clean_match, time_clean_match)

                # time_clean_match = re.sub(r'\s+', ' ', time_clean_match)
                
                # not exact raw time, but if use the index it is fine
                # print(row["tlink_inst"])
                start_ind, end_ind = _find_chemo_word_index(clean_note, matches[0], tlink_inst, before_tag_ind)
                
                # if start_ind is None or end_ind is None:
                #     print("skip here chemo: ", pat_id, filename, "chemo: ", matches)
                #     continue
                # print("skip here time: ", pat_id, filename, time_matches[0])
                time_start_ind, time_end_ind = _find_time_word_index(clean_note, time_matches[0])

                # if time_start_ind == None and time_end_ind == None:
                #     print()
                #     time_clean_match = decrease_space_before_punctuation(time_clean_match)
                #     time_start_ind, time_end_ind = _find_time_word_index(clean_note, tlink_inst, time_clean_match)
             
                chemo_start_ind_tlinkinst.append(start_ind)
                if end_ind is None:
                    chemo_end_ind_tlinkinst.append(None)
                else: 
                    chemo_end_ind_tlinkinst.append(end_ind-1)
                time_start_ind_tlinkinst.append(time_start_ind)
                if time_end_ind is None: 
                    time_end_ind_tlinkinst.append(None)
                else: 
                    time_end_ind_tlinkinst.append(time_end_ind-1)

                # temp_time1 = "".join(clean_note[time_start_ind:time_end_ind].split())
                # temp_time2 = "".join(time_matches[0].split())
                # temp_time1, temp_time2 = same_characters_in_order(temp_time1, temp_time2)

                # if (clean_note[start_ind:end_ind] != matches[0] or temp_time1 !=temp_time2:
                #     if clean_note[start_ind:end_ind] != matches[0]:
                #         print("chemo: no", pat_id, filename, start_ind, end_ind, clean_note[start_ind:end_ind], matches[0])
                        
                #     elif clean_note[time_start_ind:time_end_ind] !=time_matches[0]:
                #         print("2time: ") 
                #         print(row["chemo_annotation_id"], time_start_ind, time_end_ind, temp_time2, temp_time1, time_clean_match)

            else:
                # chemo_start_ind_tlinkinst.append(None)
                # chemo_end_ind_tlinkinst.append(None)
                # time_start_ind_tlinkinst.append(None)
                # time_end_ind_tlinkinst.append(None)
                # raw_time_lst.append(None)
                continue
            # print("yes: ", len(chemo_lst), chemo_lst)    
            nth_sent_item = _df_find_nth(chemo_lst, chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst)
            # print("chemo before: ",nth_sent_item)
            # for i in range(len(nth_sent_item)):
            #     print("chemo: ", nth_sent_item[i], chemo_lst[i], list(doc.sents)[nth_sent_item[i]])
 
            # if time_start_ind is None:
            #     time_nth_sent_item = None
            # else: 
            time_nth_sent_item = _df_find_nth(raw_time_lst, time_start_ind_tlinkinst, time_end_ind_tlinkinst)
            # for i in range(len(time_nth_sent_item)):
            #     print("time: ", time_nth_sent_item[i], raw_time_lst[i], list(doc.sents)[time_nth_sent_item[i]])


            # if nth_sent_item is None or time_nth_sent_item is None or nth_sent_item == "none" or time_nth_sent_item == "none":
            #     match_sents.append(None)
            # elif int(nth_sent_item) < int(time_nth_sent_item):
            #     match_sents.append(sent_lst[nth_sent_item:time_nth_sent_item])
            # elif int(nth_sent_item) == int(time_nth_sent_item): 
            #     match_sents.append(sent_lst[time_nth_sent_item])
            # elif int(nth_sent_item) > int(time_nth_sent_item): 
            #     match_sents.append([])
            # else:
            #     print("here: ", row)    

        if len(time_start_ind_tlinkinst) != len(nth_sent_item): 
            # chemo_start_ind_tlinkinst = []
            # chemo_end_ind_tlinkinst = []
            # nth_sent_item = []
            # time_start_ind_tlinkinst = []
            # time_end_ind_tlinkinst = []
            # time_nth_sent_item = []
            # raw_time_lst = []

            skip += 1
            print("SKIP THIS PART") 
        
        else: 

            tsv_df["chemo_tlinkinst_start"] = chemo_start_ind_tlinkinst
            tsv_df["chemo_tlinkinst_end"] = chemo_end_ind_tlinkinst
            tsv_df["nth_sent"] = nth_sent_item

            tsv_df["time_tlinkinst_start"] = time_start_ind_tlinkinst
            tsv_df["time_tlinkinst_end"] = time_end_ind_tlinkinst
            tsv_df["time_nth_sent"] = time_nth_sent_item

            tsv_df["raw_time"] = raw_time_lst
            prev_shape = tsv_df.shape
            sent_range_lst = []
            
            tsv_df.dropna()
            print(f"drop rows: {prev_shape[0] - tsv_df.shape[0]}")
            # print(f"len(time_nth_sent): ", len(time_nth_sent))
            for i in range(len(time_nth_sent_item)):
                try: 
                    if int(nth_sent_item[i]) > int(time_nth_sent_item[i]):
                        start_nth_sent = int(time_nth_sent_item[i])
                        end_nth_sent = int(nth_sent_item[i])
                    else: 
                        start_nth_sent = int(nth_sent_item[i])
                        end_nth_sent = int(time_nth_sent_item[i])
                except: 
                    print(tsv_df)
                sent_range = _find_tsv_sentences_in_range(doc, start_nth_sent, end_nth_sent+1)
                # print("ent_range: ", sent_range)
                sent_range_lst.append(sent_range) 
            # print(f"len(sent_range_lst): {len(sent_range_lst)}")
            # print(type(sent_range_lst), sent_range_lst)
            # print(type(" ".join([sent[0].text for sent in sent_range_lst]))," ".join([sent[0].text for sent in sent_range_lst]))
            tsv_df["match_sents"] = " ".join([sent[0].text for sent in sent_range_lst])
            patient_file = patient_file.reset_index()
            tsv_df = tsv_df.reset_index()
            new_df = pd.concat([patient_file, tsv_df], axis=1)
            medium_df = pd.concat([new_df, medium_df])
    final_df = pd.concat([final_df, medium_df])
final_df = final_df
# new_df_filtered = new_df[~new_df["tlink"].apply(lambda x: x.lower()).eq("none")] 
final_df.to_csv(f"./{cancer}_{mode}_expand.tsv", sep="\t", index=False)
# final_df.to_csv(f"./temp.tsv", sep="\t", index=False)

with open(f"./{cancer}_{mode}_pos_rel.json", 'w') as outfile:
    json.dump(pos_rel, outfile, indent=4)
#[["note_name", "chemo_text", "normed_timex", "raw_time", "ith_start", "ith_end", "time_ith_start", "time_ith_end", "match_sents"]]
# temp_df = final_df[["note_name", "chemo_text", "normed_timex","nth_sent", "time_nth_sent", "match_sents"]]
# temp_df.to_csv(f"./temp.tsv", sep="\t", index=False)

print(final_df.shape)
print(skip)
'''

{'patient109': {'report035_NOTE': {'raw_note': '===================================================================\nReport ID.....................1149,PBCegZLhq2zi\nPatient ID....................PBCegZLhq2zi\nPatient Name..................Patient109\nPrincipal Date................20131217\nRecord Type...................NOTE\n===================================================================\n[Report de-identified (Limited dataset compliant) by De-ID v.6.24.5.1]\n\nChief Complaint: New Patient\n\n HPI:  The patient is a 66 year old G3P3 that presents for annual gynecological examination.\n LMP: No LMP recorded. Patient is postmenopausal.\n Sexual Activity: The patient is not currently sexually active\n Any new partners since last exam? No\n Bladder control issues: None\n Current Issues/Problems: see below\n The Review of Systems was completed by the patient, reviewed, and scanned to the chart.\n Past Medical History\n Diagnosis Date\n Abnormal pap smear-cervix-unspec\n\n f/u was norma

'\ntest_all_patient_id = ""\n        # 2\n        chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst = [], []\n        nth_sent, chemo_lst= [], []\n        time_start_ind_tlinkinst, time_end_ind_tlinkinst = [], []\n        time_nth_sent,raw_time_lst = [], []\n        \n        match_sents = []\n        \n        patient_file = df.loc[df["note_name"] == pat_id + "_" + filename]\n\n        tsv_df = pd.DataFrame()\n        for i, row in patient_file.iterrows():\n            tlink_inst = row["tlink_inst"]\n            norm_time = row["normed_timex"]\n            if (tlink_inst != "none") and norm_time[0]!= "P": #why here\n                matches, before_tag_ind = _find_surrounding_text(tlink_inst)\n\n                print(pat_id, filename)\n\n                time_matches, _ = _find_time_surrounding_text(tlink_inst)\n                # if pat_id == "patient47" and filename== "report031_NOTE" and time_matches[0]=="today \'s":\n                #     continue\n                # elif time_matche

## Task 2